In [ ]:
# ==================================================
# 3일차 - 1교시: 내 모델, 얼마나 믿을 수 있을까? & 더 좋은 모델 만들기
# ==================================================
print("--- 3일차 1교시 시작 ---")

# 1. 필요한 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # 시각화에 간혹 사용될 수 있음
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import StandardScaler # 필요시 사용
%matplotlib inline

# [텍스트 셀 안내]
# (df_auto는 2일차까지의 실습을 통해 주요 숫자 컬럼 전처리가 완료되었다고 가정합니다.)
# 특히 'engine-size'와 'price' 컬럼은 숫자형이고 결측치가 처리된 상태여야 합니다.
# 만약 df_auto가 없거나 전처리가 미흡하다면, 이전 차시 코드를 먼저 실행/수정해주세요!

# ---!!! 모델 평가 및 개선에 사용할 X, Y 데이터 준비 (최종 점검) !!!---
# (이전 차시에서 X_engine_size, Y_price 등으로 이미 정의했을 수 있지만, 명확성을 위해 다시 정의)
print("\n--- 모델 평가용 X, Y 데이터 준비 (engine-size vs price) ---")

# df_auto가 이 코드 블록 실행 전에 정의되어 있고, 필요한 컬럼들이 있다고 가정합니다.
# 만약 df_auto가 정의되지 않았다면 NameError가 발생합니다.
if 'df_auto' in locals() and isinstance(df_auto, pd.DataFrame): # df_auto 존재 및 타입 확인 추가
    if 'engine-size' in df_auto.columns and 'price' in df_auto.columns and \
       pd.api.types.is_numeric_dtype(df_auto['engine-size']) and pd.api.types.is_numeric_dtype(df_auto['price']):

        X_eval = df_auto[['engine-size']].copy() # 작업용 복사본 사용
        Y_eval = df_auto['price'].copy()

        # 혹시 모를 NaN 값 제거 (train_test_split이나 cross_val_score는 NaN을 싫어함)
        # X, Y 쌍을 유지하면서 NaN 제거
        combined_eval = pd.concat([X_eval, Y_eval], axis=1).dropna()
        
        if not combined_eval.empty:
            X_eval_clean = combined_eval[['engine-size']]
            Y_eval_clean = combined_eval['price']
            print(f"NaN 제거 후 X_eval_clean shape: {X_eval_clean.shape}, Y_eval_clean shape: {Y_eval_clean.shape}")

            # 2. 학습 데이터와 테스트 데이터 분리하기 (train_test_split)
            print("\n--- 학습/테스트 데이터 분리 ---")
            X_train_d3, X_test_d3, y_train_d3, y_test_d3 = train_test_split(
                X_eval_clean, Y_eval_clean, test_size=0.3, random_state=0
            )
            print(f"X_train_d3 shape: {X_train_d3.shape}, X_test_d3 shape: {X_test_d3.shape}")

            # 3. 분리된 학습 데이터로 모델 학습 및 테스트 데이터로 평가
            print("\n--- 분리된 데이터로 학습 및 테스트 평가 ---")
            lm_split_d3 = LinearRegression()
            lm_split_d3.fit(X_train_d3, y_train_d3)
            r2_test_d3 = lm_split_d3.score(X_test_d3, y_test_d3)
            r2_train_d3 = lm_split_d3.score(X_train_d3, y_train_d3) # 학습셋 점수
            print(f"테스트 데이터 R-squared: {r2_test_d3:.4f}")
            print(f"학습 데이터 R-squared: {r2_train_d3:.4f}")
            if abs(r2_train_d3 - r2_test_d3) > 0.1: # 예시: 차이가 크면 과적합 의심
                print("학습 점수와 테스트 점수 차이가 다소 있습니다. 과적합을 확인해보세요.")

            # 4. 교차 검증으로 모델 성능 더 신뢰성 있게 평가하기 (cross_val_score)
            print("\n--- 교차 검증 (10-Fold) ---")
            lm_cv_d3 = LinearRegression()
            # 교차 검증 시에는 전체 (정제된) 데이터를 사용
            cv_scores_r2_d3 = cross_val_score(lm_cv_d3, X_eval_clean, Y_eval_clean, cv=10, scoring='r2')
            print("각 폴드별 R2 점수:", cv_scores_r2_d3)
            print(f"교차 검증 평균 R2 점수: {np.mean(cv_scores_r2_d3):.4f}")
            print(f"R2 점수 표준편차: {np.std(cv_scores_r2_d3):.4f}")

            # 5. 다항 회귀와 파이프라인을 사용한 모델 성능 개선 시도
            print("\n--- 다항 회귀 + 파이프라인 ---")
            # 2차 다항 회귀
            pipe_poly2_d3 = Pipeline([
                ("poly_feat_d2", PolynomialFeatures(degree=2, include_bias=False)),
                ("lin_reg_d2", LinearRegression())
            ])
            cv_scores_poly2_d3 = cross_val_score(pipe_poly2_d3, X_eval_clean, Y_eval_clean, cv=10, scoring='r2')
            print(f"2차 다항 회귀 CV 평균 R2: {np.mean(cv_scores_poly2_d3):.4f}")

            # 3차 다항 회귀
            pipe_poly3_d3 = Pipeline([
                ("poly_feat_d3", PolynomialFeatures(degree=3, include_bias=False)),
                ("lin_reg_d3", LinearRegression())
            ])
            cv_scores_poly3_d3 = cross_val_score(pipe_poly3_d3, X_eval_clean, Y_eval_clean, cv=10, scoring='r2')
            print(f"3차 다항 회귀 CV 평균 R2: {np.mean(cv_scores_poly3_d3):.4f}")

            print("\n모델 성능 비교:")
            print(f"  단순 선형 회귀 CV R2: {np.mean(cv_scores_r2_d3):.4f}")
            print(f"  2차 다항 회귀 CV R2: {np.mean(cv_scores_poly2_d3):.4f}")
            print(f"  3차 다항 회귀 CV R2: {np.mean(cv_scores_poly3_d3):.4f}")
            print("어떤 모델의 성능이 가장 좋았나요? 차수를 계속 높이면 어떻게 될까요?")

        else: # if not combined_eval.empty:
            print("NaN 값 제거 후 모델 평가를 위한 데이터가 남아있지 않습니다.")
            
    else: # if 'engine-size' in df_auto.columns and ...
        error_messages = []
        if 'engine-size' not in df_auto.columns:
            error_messages.append("'engine-size' 컬럼이 없습니다.")
        elif not pd.api.types.is_numeric_dtype(df_auto['engine-size']):
            error_messages.append("'engine-size' 컬럼이 숫자형이 아닙니다.")
        
        if 'price' not in df_auto.columns:
            error_messages.append("'price' 컬럼이 없습니다.")
        elif not pd.api.types.is_numeric_dtype(df_auto['price']):
            error_messages.append("'price' 컬럼이 숫자형이 아닙니다.")
        
        if error_messages:
            print("모델 평가에 필요한 데이터 조건 미충족:")
            for msg in error_messages:
                print(f"- {msg}")
        else:
            # 이 경우는 'df_auto'가 있지만 if 조건 중 일부가 False인 경우 (거의 발생 안 함)
            print("모델 평가에 필요한 'engine-size' 또는 'price' 컬럼 관련 설정에 문제가 있습니다. 데이터를 확인해주세요.")
else: # if 'df_auto' in locals() and isinstance(df_auto, pd.DataFrame):
    print("오류: df_auto DataFrame이 정의되지 않았거나 유효한 DataFrame이 아닙니다. 이전 실습을 통해 df_auto를 로드하고 전처리해주세요.")

print("\n--- 3일차 1교시 실습 종료 ---")